# Test Bubble Detection model on new test images
- Evaluates performance of custom bubble detection model on new test images
- Calculates various precision, recall values

In [20]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [22]:
# indicate custom model & desired checkpoint from training
CUSTOM_MODEL = 'my_centernet_hg104_1024_5'
CUSTOM_CHECKPOINT = 'ckpt-21'
LABEL_MAP_NAME = 'label_map.pbtxt'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
# indicate name of folder of new test images
NEW_TEST_IMGS = 'test'#'supersaturation_0.16_cropJG'
# give unique name for tf record file
NEW_RECORD_FILE_NAME = 'test_origimgs.record'

In [23]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations',CUSTOM_MODEL),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join('Tensorflow', 'workspace','images',NEW_TEST_IMGS),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [24]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [25]:
# create pipeline path for new test images
pipeline_path = os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, NEW_TEST_IMGS)
if not os.path.exists(pipeline_path):
    if os.name == 'posix':
        os.mkdir(pipeline_path)
    if os.name == 'nt':
        os.mkdir(pipeline_path)

# save path name for new pipeline config to files
files['PIPELINE_CONFIG_NEW'] = os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, NEW_TEST_IMGS,'pipeline.config')

In [26]:
# CHECKPOINT PATH: copy pipeline.config (old) into new config folder
if os.name =='posix':
    !cp {files['PIPELINE_CONFIG']} {files['PIPELINE_CONFIG_NEW']}
if os.name == 'nt':
    !copy {files['PIPELINE_CONFIG']} {files['PIPELINE_CONFIG_NEW']}

In [27]:
# ANNOTATION PATH: create tf.record file for new test images
!python {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH_TEST']} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], NEW_RECORD_FILE_NAME)} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_5/test_origimgs.record


In [28]:
# Load pipeline config (copy of old config file)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_NEW'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [29]:
# MODIFY PIPELINE.CONFIG
# indicate the path to new test image tf record file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], NEW_RECORD_FILE_NAME)]

In [31]:
# save new pipeline config (to model folder)
config_text = text_format.MessageToString(pipeline_config)
# dfile: accessing the file ('wb'= writing to file in binary mode)                                                                                                                                                                                                     
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_NEW'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [32]:
# Load pipeline config (new/modified config file)
configs_test = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG_NEW'])

In [33]:
configs_test

{'model': center_net {
   num_classes: 1
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 },
 'train_config': batch_si

# 7. Evaluate the Model
Calculate performance metrics

In [34]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [35]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG_NEW'], paths['CHECKPOINT_PATH'])

In [36]:
# command can be run from prompt
# EVAL FOLDER MUST BE EMPTY BEFORE RUNNING COMMAND! ?
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_5 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_5/test/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_5


# See Evaluation graphically with Tensorboard
- Eval folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\eval --> AV, AR
- Train folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\train --> losscurve

in prompt: go to directory

run command: tensorboard --logdir=.
    
open localhost url